Useful Python functions

In [2]:
my_string = "to be Or NOT to be"
my_list = my_string.split()
print 'String length is:', len(my_string)
print 'String contains words:', my_string.split()
print 'String contains unique words:', set(my_string.split())
print 'String to lowercase: ', my_string.lower()
print 'Array/list length (number of words) is:', len(my_list)
print 'The first item in array is:', my_list[0]
print 'The last item in array is:', my_list[-1]

String length is: 18
String contains words: ['to', 'be', 'Or', 'NOT', 'to', 'be']
String contains unique words: {'Or', 'be', 'to', 'NOT'}
String to lowercase:  to be or not to be
Array/list length (number of words) is: 6
The first item in array is: to
The last item in array is: be


## Launch the interactive shell PySpark

`pyspark --master yarn --num-executors 4`

### Example: word count

In [ ]:
def split_string(verse):    
    return verse.split(' ')

lines = sc.textFile("/user/pascepet/data/bible.txt")
words = lines.flatMap(split_string)
pairs = words.map(lambda word: (word, 1))
counts = pairs.reduceByKey(lambda a, b: a + b)
counts.take(10)

## Task 1

Adjust the above example "word count" (in any order):

1. Choose RDD which is useful to be cached and cache it.
1. Sort the results by frequency in descending order (Hint: sortBy nebo sortByKey).
1. Count the word no matter of upper- or lowercase (god, God and GOD should be same word).
1. Omit the identifying part of each line (Bible book and chapter:verse -- divided by tab \t).
1. Get rid of non-alphanumeric characters from the text ('.', ':', '-' etc. -- or at least some of them).

### Data

`/user/pascepet/data/bible.txt` at HDFS

**1. caching:**   
`words.cache()` or `words = lines.flatMap(lambda line: line.split(" ")).cache()`

**2. sorting results:**  
`counts_sorted = counts.sortBy(lambda x: x[1], ascending=False)`

**3. case-insensitivity -- e. g. transforming words to lowercase:**  
`words = words.map(lambda word: word.lower())`

**4. separating start of row (omitting row identification):**  
`lines = lines.map(lambda line: line.split("\t")[1])`

**5. removing of non-alphanumeric characters:**  
* removing of a particular non-alphanum. character:  
`words = words.map(lambda word: word.replace('.', '')`
* removing of all non-alphanum. character:  
`import re`  
`words = words.map(lambda word: re.sub(r'\W+', '', word))`

## Task 2

a) Count words in each verse (1 verse = 1 line) and find the verses with the highest and lowest number of words.  
b) Do the same calculation as in a) but count only unique words in each verse.

### Data

`/user/pascepet/bible.txt` na HDFS

### Expected result

| verse_id | word_count |  
| -------- | ----------:|  

----

In [ ]:
# this code contains solution for a) and b) together
# reading RDD
lines = sc.textFile("/user/pascepet/data/bible.txt") 

# splitting a row to id and text
lines2 = lines.map(lambda line: line.split("\t"))

# creating a pair (row id, list of words), caching
lines3 = lines2.map(lambda line: (line[0], line[1].split(" "))).cache()
# it's possible to clean words like in Task 1: to lowercase, removing of non-alphanumeric etc.

# counting words and unique words
counts = lines3.map(lambda line: (line[0], len(line[1]))).cache()
counts_uniq = lines3.map(lambda line: (line[0], len(set(line[1])))).cache()

# sorting rows by word count, printing first element
counts_sorted_asc = counts.sortBy(lambda x: x[1], True)
counts_sorted_desc = counts.sortBy(lambda x: x[1], False)
counts_sorted_asc.take(1)
counts_sorted_desc.take(1)
counts_uniq_sorted_asc = counts_uniq.sortBy(lambda x: x[1], True)
counts_uniq_sorted_desc = counts_uniq.sortBy(lambda x: x[1], False)
counts_uniq_sorted_asc.take(1)
counts_uniq_sorted_desc.take(1)

## Task 3

a) Get the state with the highest average temperature over months 6--8. Write the results in Celsius degrees.  
b) For each month get the state with the highest average temperature in that months.

### Data

1. Find the file `/home/pascepet/fel_bigdata/data/teplota-usa.zip` on the local filesystem (not at HDFS).
1. Copy this file to your working directory on the local filesystem.
1. Unzip the file. You will get two files CSV. Join them into one file `teplota.csv`.
1. The file `teplota.csv` copy to HDFS to your user directory.

### Data description

CSV file is delimited by ','. It has headers with column names which we need to filter out.        
The temperature is in 10 * Fahrenheit degrees.     
Some row have no temperature measured (empty string).

**Columns:** id_station, month, day, hour, temperature, flag, latitude, longitude, hight, state, name


### Expected result

In a) section

| stat | avg_temperature |  
| ---- | ---------------:|  

In b) section

| month | state | avg_temperature |  
| ----- | ----- | ---------------:|  

----

In [ ]:
import re

# reading RDD, removing (filtering out) unwanted rows
# !!! use the path to your own user directory at HDFS
lines = sc.textFile("/user/pascepet/data/teplota.csv")
lines2 = lines.filter(lambda line: not(re.match(r'stanice', line)))

# transforming a row to the structure ((month, state), temperature)
recs = lines2.map(lambda line: line.split(","))
# removing rows with empty temperature
recs = recs.filter(lambda rec: rec[4]!='')
# Fahrenheita to Celsius conversion
recs = recs.map(lambda rec: ((int(rec[1]), rec[9]), (float(rec[4])/10-32)*5/9))
# caching
recs = recs.persist(StorageLevel.MEMORY_AND_DISK)

# a) state with the highest average temperature in June--August
# removing data of other months
recs1 = recs.filter(lambda rec: rec[0][0] in range(6,9))
# keeping state, omitting month, preparing pairs for aggregation
recs1 = recs1.map(lambda rec: (rec[0][1], (1, rec[1])))
result1 = recs1.reduceByKey(lambda s,t: (s[0]+t[0], s[1]+t[1]))  # aggregation by states - count and sum
result1 = result1.map(lambda res: (res[0], res[1][1]/res[1][0])) # average = sum/count
result1 = result1.sortBy(lambda res: res[1], False)
result1.take(1)

# b) for each month: state with the highest average temperature
# calculating aggregation for each pair (month, state)
recs2 = recs.map(lambda rec: (rec[0], (1, rec[1])))
result2 = recs2.reduceByKey(lambda s,t: (s[0]+t[0], s[1]+t[1]))
result2 = result2.map(lambda res: (res[0], res[1][1]/res[1][0])) # now we have the average temperature for each pair (month, state)
result2 = result2.map(lambda res: (res[0][0], (res[0][1], res[1]))) # we use month as a key and search maximum over states 
result3 = result2.reduceByKey(lambda s,t: (t if t[1]>s[1] else s))
result3 = result3.sortBy(lambda res: res[0], True)
result3.collect()